In [3]:
# 新しく書き直し

# raw fluor, neuropil, deconvの波形からneuronを抽出
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import loadmat, savemat
import tifffile
import cv2
import random
# from setGUIFont import *
import datetime
import time

from PyQt5.QtWidgets import *
from PyQt5.QtGui import QPixmap, QImage, QPainter, QPen, QColor, QFont, QPainterPath, QBrush
from PyQt5.QtCore import Qt, QTimer, QItemSelection
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure

dir_notebook = os.path.dirname(os.path.abspath("__file__"))
# 親ディレクトリのパスを取得
dir_parent = os.path.dirname(dir_notebook)
if not dir_parent in sys.path:
    sys.path.append(dir_parent)

from optic.config import *
from optic.gui import *
from optic.io import *
from optic.utils import *
from optic.preprocessing import *

# アプリ名の設定
AppSettings.setCurrentApp("SUITE2P_ROI_CHECK")

class Suite2pROICheckGUI(QMainWindow, WidgetManager):
    def __init__(self):
        """
        pri: 格納するkey
        """
        QMainWindow.__init__(self)
        WidgetManager.__init__(self)
        # self.class_HoloStimResponse = None

        # Tableの列名と列番号, 列情報のdict
        self.dict_tablecol = AppSettings.getTableColumns()
        # キーと機能のマッピング
        self.key_function_map = AppSettings.getKeyFunctionMap()
        # GUIのコンフィグ
        self.gui_defaults = AppSettings.getGuiDefaults()

        self.setWindowTitle(self.gui_defaults["TITLE"])
        self.setGeometry(self.gui_defaults["INIT_POSITION_X"], self.gui_defaults["INIT_POSITION_Y"], self.gui_defaults["WIDTH"], self.gui_defaults["HEIGHT"])
        self.setupUI_done = False
        
        # self.initializeAttributes()

        # self.setupFileLoadUI()

if __name__ == "__main__":
    app = QApplication(sys.argv) if QApplication.instance() is None else QApplication.instance()
#     font = setGUIFont(app)
    gui = Suite2pROICheckGUI()
    gui.show()
    sys.exit(app.exec_())

SystemExit: 0

c:\Users\HyperPC_Smee\anaconda3\envs\itkelastix\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
gui.gui_defaults

{'ROI_OPACITY': 50,
 'SELECTED_ROI_OPACITY': 255,
 'MIN_PLOT_RANGE': 30,
 'INIT_POSITION_X': 100,
 'INIT_POSITION_Y': 100,
 'WIDTH': 1200,
 'HEIGHT': 200,
 'TITLE': 'Suite2pROICheckGUI'}